# Tanzania Wells Analysis
## by Allison Ward, Sarah Prusaitis, and Monica Pecha

Business problem: Unicef is looking to asses the condition of hand pump and communal standpipe wells in the Lake Victoria basin. They have requested that we help them predict the likelihood of a well being functional, in need of repair, or nonfunctional.

The dataset was taken from [Taarifa](https://taarifa.org/) and the [Tanzanian Ministry of Water](https://www.maji.go.tz/). 

# Exploratory Data Analysis

In [1]:
import pandas as pd
import numpy as np

In [3]:
# Reading in our datasets to explore
df1 = pd.read_csv('data/test_set_values.csv')
df2= pd.read_csv('data/training_set_labels.csv')
df3 = pd.read_csv('data/training_set_values.csv')

In [ ]:
df1.head()

In [ ]:
df1.info()

In [ ]:
df2.head()

In [ ]:
df2.info()

In [ ]:
df3.head()

In [ ]:
df3.info()

In [5]:
# dropping irrelevant columns
to_drop = ["scheme_name", "public_meeting", "payment_type", "quantity_group", "waterpoint_type", "amount_tsh", "region_code", "district_code", "num_private", "date_recorded", "recorded_by"]
df3.drop(to_drop, axis=1, inplace = True)

In [6]:
# merging dataframes 2 and 3 on id
df4 = df3.merge(df2, on = "id")
df4.head()

,id,funder,gps_height,installer,longitude,latitude,wpt_name,basin,subvillage,region,...,management_group,payment,water_quality,quality_group,quantity,source,source_type,source_class,waterpoint_type_group,status_group
0,69572,Roman,1390,Roman,34.938093,-9.856322,none,Lake Nyasa,Mnyusi B,Iringa,...,user-group,pay annually,soft,good,enough,spring,spring,groundwater,communal standpipe,functional
1,8776,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,Lake Victoria,Nyamara,Mara,...,user-group,never pay,soft,good,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,functional
2,34310,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,Pangani,Majengo,Manyara,...,user-group,pay per bucket,soft,good,enough,dam,dam,surface,communal standpipe,functional
3,67743,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,Ruvuma / Southern Coast,Mahakamani,Mtwara,...,user-group,never pay,soft,good,dry,machine dbh,borehole,groundwater,communal standpipe,non functional
4,19728,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,Lake Victoria,Kyanyamisa,Kagera,...,other,never pay,soft,good,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,functional


In [7]:
# dropping any wells that are not located in the Lake Victoria basin (our stakeholder wants to focus on one region)
df4.drop(df4.index[df4['basin'] != 'Lake Victoria'], inplace = True)
df4.head()

,id,funder,gps_height,installer,longitude,latitude,wpt_name,basin,subvillage,region,...,management_group,payment,water_quality,quality_group,quantity,source,source_type,source_class,waterpoint_type_group,status_group
1,8776,Grumeti,1399,GRUMETI,34.698766,-2.147466e+00,Zahanati,Lake Victoria,Nyamara,Mara,...,user-group,never pay,soft,good,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,functional
4,19728,Action In A,0,Artisan,31.130847,-1.825359e+00,Shuleni,Lake Victoria,Kyanyamisa,Kagera,...,other,never pay,soft,good,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,functional
9,46144,Isingiro Ho,0,Artisan,30.626991,-1.257051e+00,Kwapeto,Lake Victoria,Mkonomre,Kagera,...,user-group,never pay,soft,good,enough,shallow well,shallow well,groundwater,hand pump,functional
18,34169,Hesawa,1162,DWE,32.920154,-1.947868e+00,Ngomee,Lake Victoria,Center,Mwanza,...,user-group,never pay,milky,milky,insufficient,spring,spring,groundwater,other,functional needs repair
21,6091,Dwsp,0,DWE,0.000000,-2.000000e-08,Muungano,Lake Victoria,Ibabachegu,Shinyanga,...,user-group,unknown,unknown,unknown,unknown,shallow well,shallow well,groundwater,hand pump,functional


In [8]:
# dropping any wells that are not a communal standpipe or hand pump
filtered_df = df4[df4['waterpoint_type_group'].isin(['communal standpipe', 'hand pump'])]

In [9]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8330 entries, 1 to 59385
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     8330 non-null   int64  
 1   funder                 8214 non-null   object 
 2   gps_height             8330 non-null   int64  
 3   installer              8215 non-null   object 
 4   longitude              8330 non-null   float64
 5   latitude               8330 non-null   float64
 6   wpt_name               8330 non-null   object 
 7   basin                  8330 non-null   object 
 8   subvillage             8326 non-null   object 
 9   region                 8330 non-null   object 
 10  lga                    8330 non-null   object 
 11  ward                   8330 non-null   object 
 12  population             8330 non-null   int64  
 13  scheme_management      7515 non-null   object 
 14  permit                 7987 non-null   object 
 15  construc

In [11]:
filtered_df['status_group'].value_counts()

status_group
functional                 4527
non functional             2967
functional needs repair     836
Name: count, dtype: int64